## Install and import packages

In [1]:
# !pip install soundfile
# !pip install pydub
# !pip install SpeechRecognition
# !pip install openai --upgrade

In [2]:
import os
import csv
import wave
import soundfile
import requests
import pandas as pd
from datetime import timedelta
from pydub import AudioSegment
import speech_recognition as sr
from transformers import pipeline
from openai import AzureOpenAI
from dotenv import load_dotenv
from tqdm import tqdm


# Load environment variables from .env file
load_dotenv()

# Get the Hugging Face API key from the environment variables
HF_API_KEY = os.environ["HF_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

/Users/kw/Documents/py_projects/canto-text/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 
#### Setup Device (CUDA, MPS, CPU)

In [3]:
import torch

# Determine the best available device (CUDA if available, otherwise MPS, then CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [4]:
input_m4a_audio = "input_audio/1hr_sample.m4a"

---
## Convert audio to wav format

In [5]:
from pydub import AudioSegment
import os

def convert_audio_to_wav(input_file):
    """Convert .m4a file to .wav format."""
    # Load the .m4a file
    audio = AudioSegment.from_file(input_file, format="m4a")
    output_file = os.path.join("converted_audio", os.path.basename(input_file).replace(".m4a", ".wav"))
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    audio.export(output_file, format="wav")
    
    return output_file

In [6]:
convert_audio_to_wav(input_m4a_audio)

'converted_audio/1hr_sample.wav'

---
## Audio Chunking
### using pyannote

In [7]:
import os
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import medfilt

def split_audio_by_silence(audio_path, output_dir, chunk_duration=300, min_silence_duration=0.5, 
                           silence_threshold=-40, padding=0.2):
    """
    Split an audio file into chunks based on silence detection to avoid splitting in the middle of sentences.
    
    Parameters:
    - audio_path: Path to the input audio file
    - output_dir: Directory to save the output chunks
    - chunk_duration: Target duration for each chunk in seconds (default: 300s = 5 minutes)
    - min_silence_duration: Minimum duration of silence to consider as a potential split point (seconds)
    - silence_threshold: Threshold in dB for determining silence
    - padding: Padding time in seconds to add to each chunk to ensure overlap
    
    Returns:
    - List of paths to the generated audio chunks
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the audio file with pydub
    audio = AudioSegment.from_file(audio_path)
    
    # Convert to mono if stereo for easier processing
    if audio.channels > 1:
        audio = audio.set_channels(1)
    
    # Get the duration in milliseconds
    total_duration_ms = len(audio)
    
    # Convert chunk duration to milliseconds
    chunk_duration_ms = chunk_duration * 1000
    
    # Load the audio file with librosa for silence detection
    y, sr = librosa.load(audio_path, sr=None)
    
    # Compute the RMS energy
    rms = librosa.feature.rms(y=y, frame_length=int(sr * 0.025), hop_length=int(sr * 0.01))[0]
    
    # Convert to dB
    rms_db = 20 * np.log10(rms + 1e-10)
    
    # Apply median filtering to smooth the energy curve
    rms_db_smoothed = medfilt(rms_db, kernel_size=11)
    
    # Find silence regions (below threshold)
    silence_mask = rms_db_smoothed < silence_threshold
    
    # Convert to time (seconds)
    times = librosa.times_like(rms_db_smoothed, sr=sr, hop_length=int(sr * 0.01))
    
    # Find continuous silence regions
    silence_regions = []
    in_silence = False
    start_time = 0
    
    for i, is_silent in enumerate(silence_mask):
        if is_silent and not in_silence:
            # Start of a silence region
            start_time = times[i]
            in_silence = True
        elif not is_silent and in_silence:
            # End of a silence region
            end_time = times[i]
            silence_duration = end_time - start_time
            if silence_duration >= min_silence_duration:
                silence_regions.append((start_time, end_time))
            in_silence = False
    
    # Add the last silence region if we're still in silence at the end
    if in_silence:
        end_time = times[-1]
        silence_duration = end_time - start_time
        if silence_duration >= min_silence_duration:
            silence_regions.append((start_time, end_time))
    
    # Sort silence regions by their start time
    silence_regions.sort(key=lambda x: x[0])
    
    # Determine the split points
    split_points = []
    current_chunk_end_ms = chunk_duration_ms
    
    # Find the closest silence region to each target chunk end
    for i in range(len(silence_regions)):
        silence_start_ms = silence_regions[i][0] * 1000
        silence_end_ms = silence_regions[i][1] * 1000
        
        # If this silence region is close to our target chunk end
        if abs(silence_start_ms - current_chunk_end_ms) < chunk_duration_ms * 0.2:
            split_points.append(silence_start_ms)
            current_chunk_end_ms += chunk_duration_ms
        
        # If we've already passed our target chunk end, use this silence region
        elif silence_start_ms > current_chunk_end_ms:
            split_points.append(silence_start_ms)
            current_chunk_end_ms = silence_start_ms + chunk_duration_ms
    
    # Add the start of the audio as the first split point
    split_points = [0] + split_points
    
    # Add the end of the audio as the last split point
    if split_points[-1] < total_duration_ms:
        split_points.append(total_duration_ms)
    
    # Generate chunks
    chunk_paths = []
    
    for i in tqdm(range(len(split_points) - 1), desc="Splitting audio into chunks"):
        # Calculate start and end times with padding
        start_ms = max(0, split_points[i] - padding * 1000)
        end_ms = min(total_duration_ms, split_points[i + 1] + padding * 1000)
        
        # Extract the chunk
        chunk = audio[start_ms:end_ms]
        
        # Generate the output filename
        output_filename = f"chunk_{i+1:03d}.wav"
        output_path = os.path.join(output_dir, output_filename)
        
        # Export the chunk
        chunk.export(output_path, format="wav")
        chunk_paths.append(output_path)
        
        # Print progress
        # print(f"Chunk {i+1}/{len(split_points)-1} - Duration: {(end_ms-start_ms)/1000:.2f}s")
    
    # Add a tqdm progress bar for chunk generation
    for i in tqdm(range(len(split_points) - 1), desc="Generating audio chunks"):
        # Calculate start and end times with padding
        start_ms = max(0, split_points[i] - padding * 1000)
        end_ms = min(total_duration_ms, split_points[i + 1] + padding * 1000)
        
        # Extract the chunk
        chunk = audio[start_ms:end_ms]
        
        # Generate the output filename
        output_filename = f"chunk_{i+1:03d}.wav"
        output_path = os.path.join(output_dir, output_filename)
        
        # Export the chunk
        chunk.export(output_path, format="wav")
        chunk_paths.append(output_path)
        
        # Print progress
        print(f"Chunk {i+1}/{len(split_points)-1} - Duration: {(end_ms-start_ms)/1000:.2f}s")
    
    return chunk_paths


In [8]:
raw_wav_file = input_m4a_audio.replace('input_audio', 'converted_audio').replace('.m4a', '.wav')
file_name = input_m4a_audio.split('/')[-1].split('.')[0]

In [9]:
chunk_paths = split_audio_by_silence(raw_wav_file, f"chunked_audio/{file_name}", chunk_duration=60)

Generating audio chunks: 100%|██████████| 59/59 [00:00<00:00, 451.21it/s]


Chunk 1/59 - Duration: 78.61s
Chunk 2/59 - Duration: 64.14s
Chunk 3/59 - Duration: 48.46s
Chunk 4/59 - Duration: 68.48s
Chunk 5/59 - Duration: 70.02s
Chunk 6/59 - Duration: 57.94s
Chunk 7/59 - Duration: 75.59s
Chunk 8/59 - Duration: 55.66s
Chunk 9/59 - Duration: 86.63s
Chunk 10/59 - Duration: 49.84s
Chunk 11/59 - Duration: 61.54s
Chunk 12/59 - Duration: 142.41s
Chunk 13/59 - Duration: 110.91s
Chunk 14/59 - Duration: 57.07s
Chunk 15/59 - Duration: 61.31s
Chunk 16/59 - Duration: 73.12s
Chunk 17/59 - Duration: 51.50s
Chunk 18/59 - Duration: 55.01s
Chunk 19/59 - Duration: 87.36s
Chunk 20/59 - Duration: 51.63s
Chunk 21/59 - Duration: 60.37s
Chunk 22/59 - Duration: 67.12s
Chunk 23/59 - Duration: 53.65s
Chunk 24/59 - Duration: 66.16s
Chunk 25/59 - Duration: 55.94s
Chunk 26/59 - Duration: 58.08s
Chunk 27/59 - Duration: 63.20s
Chunk 28/59 - Duration: 65.99s
Chunk 29/59 - Duration: 80.94s
Chunk 30/59 - Duration: 116.95s
Chunk 31/59 - Duration: 78.14s
Chunk 32/59 - Duration: 53.94s
Chunk 33/59 - 

---

## Speech (Cantonese) to Text (Spoken Cantonese)
### Method 1: using Google Gemini 1.5 pro

In [10]:
def get_all_file_paths(directory):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):  # Make sure to only include .wav files
                file_paths.append(os.path.join(root, file))
    
    # Sort the file paths based on the numeric part of the filename
    file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1]))
    
    return file_paths

In [11]:
audio_chunk_paths = get_all_file_paths(f'chunked_audio/{file_name}')
print(audio_chunk_paths[:5])

['chunked_audio/1hr_sample/chunk_001.wav', 'chunked_audio/1hr_sample/chunk_002.wav', 'chunked_audio/1hr_sample/chunk_003.wav', 'chunked_audio/1hr_sample/chunk_004.wav', 'chunked_audio/1hr_sample/chunk_005.wav']


#### Using Gemini API

In [12]:
import google.generativeai as genai

# Gemini transcription
def transcribe_with_gemini(audio_chunk_paths, api_key):
    try:
        print("Transcribing with Gemini Pro 1.5...")
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')
        
        # Create prompt for diarization and Cantonese transcription
        prompt = """
        Transcribe this Cantonese audio in Hong Kong colloquial speech, removing filler words and repeated phrases. Extract only spoken content and ignore background noise.
        """
        
        transcriptions = []
        for i, chunk_path in enumerate(tqdm(audio_chunk_paths, desc="Transcribing chunks")):
            # Upload audio file with specified MIME type
            f = genai.upload_file(chunk_path, mime_type='audio/wav')
            
            # Generate using multimodal capabilities
            response = model.generate_content([prompt, f])
            transcriptions.append(response.text)
        
        return "".join(transcriptions)
    except Exception as e:
        print(f"Error with Gemini transcription: {str(e)}")
        return None

In [15]:
audio_chunk_paths

['chunked_audio/1hr_sample/chunk_001.wav',
 'chunked_audio/1hr_sample/chunk_002.wav',
 'chunked_audio/1hr_sample/chunk_003.wav',
 'chunked_audio/1hr_sample/chunk_004.wav',
 'chunked_audio/1hr_sample/chunk_005.wav',
 'chunked_audio/1hr_sample/chunk_006.wav',
 'chunked_audio/1hr_sample/chunk_007.wav',
 'chunked_audio/1hr_sample/chunk_008.wav',
 'chunked_audio/1hr_sample/chunk_009.wav',
 'chunked_audio/1hr_sample/chunk_010.wav',
 'chunked_audio/1hr_sample/chunk_011.wav',
 'chunked_audio/1hr_sample/chunk_012.wav',
 'chunked_audio/1hr_sample/chunk_013.wav',
 'chunked_audio/1hr_sample/chunk_014.wav',
 'chunked_audio/1hr_sample/chunk_015.wav',
 'chunked_audio/1hr_sample/chunk_016.wav',
 'chunked_audio/1hr_sample/chunk_017.wav',
 'chunked_audio/1hr_sample/chunk_018.wav',
 'chunked_audio/1hr_sample/chunk_019.wav',
 'chunked_audio/1hr_sample/chunk_020.wav',
 'chunked_audio/1hr_sample/chunk_021.wav',
 'chunked_audio/1hr_sample/chunk_022.wav',
 'chunked_audio/1hr_sample/chunk_023.wav',
 'chunked_a

In [13]:
# Test with a single file
# audio_chunk_paths = ["converted_audio/sample_audio_1min.wav"]
gemini_output = transcribe_with_gemini(audio_chunk_paths, GEMINI_API_KEY)
print(gemini_output)

Transcribing with Gemini Pro 1.5...


Transcribing chunks: 100%|██████████| 59/59 [19:40<00:00, 20.01s/it]

我跟著佢哋嗰個表格寫，即係關於上年教師發放嗰陣，好似呢個，一條龍申請，只有呢，你而家deadline，即係今年其實deadline，因為一月三十一號嘛。我都唔記得，咁即係今年27號呢啲文件我喇。只有呢，嗰年嘅一月一號，佢哋假期年都有㗎。不過我再補充多次呀，年年都有呢個申請。係嗰年一月一號，佢哋今年嘅一月三十一號之前你已經完成晒成個course，然後有單有證書，返嚟拎錢。例子，讀，Miss，你未讀啊？讀咗喇。讀咗，攞咗張證書？未攞。有收據？有收據。張證書？係。辦妥先。好，申請咗先。好，申請我可唔可以點樣辦妥，即係發放。因為申請咗嘛，因為過咗如果唔再申請呢，就點啊？就過咗囉。因為佢下個年度會再計，即係下年度2025年一月一號囉，去到，因為你嘅收據係嗰年之前嘅，咁就申請唔到囉。係啦，你而家申請係最尾嘛。因為我下個月就係個deadline囉。哦，係啦係咪呀，但係你收據已經係讀書嗰陣時嘅喇。係呀。咁所以呢，就申請呢個日子先，佢計數先計呢個日子架，呢個日子。
廿八幾廿八。咁平嘅咩？係呀。因為我哋做緊特價呀！吓！係呀！我買咗幾次。廿八幾廿八唔係呀，係廿八蚊幾呀！係呀，佢哋本身都唔係好嘅筆記，因為同嗰啲質素有關嘅，但係我哋講錯咗，幾十蚊嗰啲嘢，唔係幾十蚊，係嗰個牌子嗰啲。幾百蚊嗰啲？你好似好叻喎。好呀。除非你做校長，就可以控制到。吓！你咪識嗰啲囉！係呀。其他老師都唔係咁易堅持到呀？係呀，點解？因為你小朋友好似呀老師，我年年都寫啦。係咪做咗又做咗又做，不過後尾老師話唔好再做。做返呢啲自己，因為成本咁貴，都係呀！都係咁話啦！最主要係事實囉！呢啲老師唔係好肯指正學生，真係好難教㗎嘛。當有咩情況之下，你控制唔到學生，真係好難，心煩都唔煩到我哋！係呀，以前呢以前以前舊式嗰啲係，而家呢一個年代呀，所以呢我哋教小朋友，咁你就真係㗎，唔係講笑，係要呢，逼。
之前嗰啲唔係呢個價錢買嘅。
其實個標價係呢個價錢。
係喎，佢哋加咗價錢喎。
嗰陣時我哋入貨唔係呢個價錢。
加咗幾時呀？
佢哋呢個星期先加。
所以囉，有啲嘢加幾十蚊。
都有，都有啲加幾蚊啫。
最重要係呢個款而家冇晒啦，冇晒啦，就因為而家興。
我哋呢邊都冇啦，係呀？
你哋嗰邊應該差唔多全部都冇喇而家。
冇晒喇。
呢幾日呢，我哋呢邊呀，瘋狂咁賣。
吓，真係㗎？
真係㗎，真係㗎。
唔打緊，唔打緊，我哋叫做朋友價咋。
叫做朋友價。
噉我自己…
噉我…
快

#### Save transcribed output

In [17]:
import os
import pickle

# Ensure the text_output directory exists
os.makedirs("text_output", exist_ok=True)

# Save it as a pickle file
with open(f"text_output/{file_name}_raw_transcription.pkl", "wb") as file:
    pickle.dump(gemini_output, file)

# Save it as a text file
with open(f"text_output/{file_name}_raw_transcription.txt", "w", encoding="utf-8") as text_file:
    if isinstance(gemini_output, str):
        text_file.write(gemini_output)
    elif isinstance(gemini_output, list):
        text_file.write("\n".join(gemini_output))
    else:
        text_file.write(str(gemini_output))

print("Variable pickled and saved successfully.")
print("Transcription saved to text file successfully.")

Variable pickled and saved successfully.
Transcription saved to text file successfully.


### Define Generic Gemini prompting function

In [18]:
import google.generativeai as genai

def gemini_prompt_call(message, api_key, prompt, temperature=0.3, top_p=0.8):
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')

        prompt_parts = [prompt+'\n', message]

        generation_config = genai.GenerationConfig(
                            temperature=temperature,
                            top_p=top_p,
                            )  

        response = model.generate_content(prompt_parts, generation_config=generation_config)
        return response.text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Add punctuation and refine the Spoken Cantonese Text
Using Gemini 1.5 pro

In [19]:
def refine_cantonese(message, api_key):
    prompt = """
            You are an expert in Cantonese. Please perform the following two steps and return the revised text.
            1. Correct potential inaccuracies in the Cantonese text in Hong Kong style. For example, in Hong Kong, text such as '噉', should be written as '咁'.
            2. Refine punctuation to the text.
            3. Remove repeated and unnecessary words.  For example, '係嘅係嘅係嘅' or '啱嘅啱嘅啱嘅' can be replaced with '係'.
            """
    return gemini_prompt_call(message, api_key, prompt)

In [20]:
gemini_output

'我跟著佢哋嗰個表格寫，即係關於上年教師發放嗰陣，好似呢個，一條龍申請，只有呢，你而家deadline，即係今年其實deadline，因為一月三十一號嘛。我都唔記得，咁即係今年27號呢啲文件我喇。只有呢，嗰年嘅一月一號，佢哋假期年都有㗎。不過我再補充多次呀，年年都有呢個申請。係嗰年一月一號，佢哋今年嘅一月三十一號之前你已經完成晒成個course，然後有單有證書，返嚟拎錢。例子，讀，Miss，你未讀啊？讀咗喇。讀咗，攞咗張證書？未攞。有收據？有收據。張證書？係。辦妥先。好，申請咗先。好，申請我可唔可以點樣辦妥，即係發放。因為申請咗嘛，因為過咗如果唔再申請呢，就點啊？就過咗囉。因為佢下個年度會再計，即係下年度2025年一月一號囉，去到，因為你嘅收據係嗰年之前嘅，咁就申請唔到囉。係啦，你而家申請係最尾嘛。因為我下個月就係個deadline囉。哦，係啦係咪呀，但係你收據已經係讀書嗰陣時嘅喇。係呀。咁所以呢，就申請呢個日子先，佢計數先計呢個日子架，呢個日子。\n廿八幾廿八。咁平嘅咩？係呀。因為我哋做緊特價呀！吓！係呀！我買咗幾次。廿八幾廿八唔係呀，係廿八蚊幾呀！係呀，佢哋本身都唔係好嘅筆記，因為同嗰啲質素有關嘅，但係我哋講錯咗，幾十蚊嗰啲嘢，唔係幾十蚊，係嗰個牌子嗰啲。幾百蚊嗰啲？你好似好叻喎。好呀。除非你做校長，就可以控制到。吓！你咪識嗰啲囉！係呀。其他老師都唔係咁易堅持到呀？係呀，點解？因為你小朋友好似呀老師，我年年都寫啦。係咪做咗又做咗又做，不過後尾老師話唔好再做。做返呢啲自己，因為成本咁貴，都係呀！都係咁話啦！最主要係事實囉！呢啲老師唔係好肯指正學生，真係好難教㗎嘛。當有咩情況之下，你控制唔到學生，真係好難，心煩都唔煩到我哋！係呀，以前呢以前以前舊式嗰啲係，而家呢一個年代呀，所以呢我哋教小朋友，咁你就真係㗎，唔係講笑，係要呢，逼。\n之前嗰啲唔係呢個價錢買嘅。\n其實個標價係呢個價錢。\n係喎，佢哋加咗價錢喎。\n嗰陣時我哋入貨唔係呢個價錢。\n加咗幾時呀？\n佢哋呢個星期先加。\n所以囉，有啲嘢加幾十蚊。\n都有，都有啲加幾蚊啫。\n最重要係呢個款而家冇晒啦，冇晒啦，就因為而家興。\n我哋呢邊都冇啦，係呀？\n你哋嗰邊應該差唔多全部都冇喇而家。\n冇晒喇。\n呢幾日呢，我哋呢邊呀，瘋狂咁賣。\n吓，真係㗎？\n真係㗎，真係㗎。\n唔打緊，唔打緊，我哋叫做朋友價咋。\

In [21]:
refined_spoken_text = refine_cantonese(gemini_output, GEMINI_API_KEY)

In [22]:
refined_spoken_text

'跟著佢哋個表格寫，即係關於上年教師發放嗰陣，好似呢個，一條龍申請。你而家deadline，即係今年deadline，因為一月三十一號嘛。我都唔記得，咁即係今年27號呢啲文件我喇。嗰年嘅一月一號，佢哋假期年年都有㗎。我再補充多次，年年都有呢個申請。係嗰年一月一號至今年嘅一月三十一號之前，你已經完成晒成個course，然後有單有證書，返嚟拎錢。例子，讀，Miss，你未讀呀？讀咗喇。讀咗，攞咗張證書？未攞。有收據？有。張證書？係。辦妥先申請。好，申請咗先。好，申請咗，我可唔可以點樣辦妥，即係發放？因為申請咗嘛，如果唔再申請呢，就點呀？就過咗囉。因為佢下個年度會再計，即係下年度2025年一月一號囉，去到，因為你嘅收據係嗰年之前嘅，咁就申請唔到囉。係啦，你而家申請係最尾喇嘛。因為我下個月就係deadline囉。哦，係咪呀？但係你收據已經係讀書嗰陣時嘅喇。係呀。咁所以呢，就申請呢個日子先，佢計數先計呢個日子㗎。\n\n廿八蚊幾。咁平嘅咩？係呀。因為我哋做緊特價呀！吓！係呀！我買咗幾次。係呀，佢哋本身都唔係好嘅筆記，因為同質素有關嘅，但係我哋講錯咗，幾十蚊嗰啲嘢，唔係幾十蚊，係嗰個牌子嗰啲。幾百蚊嗰啲？你好似好叻喎。好呀。除非你做校長，就可以控制到。吓！你咪識嗰啲囉！係呀。其他老師都唔係咁易堅持到呀？係呀，點解？因為你小朋友好似呀老師，我年年都寫啦。係咪做咗又做，不過後尾老師話唔好再做。做返呢啲自己，因為成本咁貴，都係呀！都係咁話啦！最主要係事實囉！呢啲老師唔係好肯指正學生，真係好難教㗎嘛。當有咩情況之下，你控制唔到學生，真係好難，心煩都唔煩到我哋！係呀，以前舊式嗰啲係，而家呢一個年代呀，所以呢我哋教小朋友，咁你就真係㗎，唔係講笑，係要逼。\n\n之前嗰啲唔係呢個價錢買嘅。\n其實個標價係呢個價錢。\n係喎，佢哋加咗價錢喎。\n嗰陣時我哋入貨唔係呢個價錢。\n加咗幾時呀？\n佢哋呢個星期先加。\n所以囉，有啲嘢加幾十蚊。\n都有，都有啲加幾蚊啫。\n最重要係呢個款而家冇晒啦，就因為而家興。\n我哋呢邊都冇啦，係呀？\n你哋嗰邊應該差唔多全部都冇喇而家。\n冇晒喇。\n呢幾日呢，我哋呢邊呀，瘋狂咁賣。\n吓，真係㗎？\n真係㗎。\n唔打緊，我哋叫做朋友價咋。\n\n咁我…\n快啲，令到自己成家好，咁個個都係能夠…\n삐——\n佢幫到小朋友咁嘛，真係學到咁嘅嘢。係呀，咁所以大家傾吓

## Speech (Cantonese) to Text (Written Chinese)
Using Gemini 1.5 pro

In [37]:
def speech_to_written(message, api_key):
    prompt = """
            Convert the following spoken Cantonese text into standard written Chinese in traditional characters (zh-Hant). Maintain grammatical correctness and coherence while ensuring the meaning is preserved. Use formal written Chinese (書面語) rather than colloquial Cantonese (口語).
            """
    return gemini_prompt_call(message, api_key, prompt)

In [38]:
raw_written = speech_to_written(refined_spoken_text, GEMINI_API_KEY)

In [39]:
raw_written

'跟著他們的表格寫，即是關於上年教師發放款項的時候，像這個，一條龍申請。你現在的截止日期，即是今年的截止日期，因為是一月三十一日嘛。我都忘記了，那麼即是今年二十七號左右我收到這些文件。當年的 一月一日，他們假期年年都有的。我再補充一次，年年都有這個申請。是當年的 一月一日至今年的 一月三十一日之前，你已經完成整個課程，然後有收據有證書，回來領錢。例子，讀，Miss，你還沒讀嗎？讀完了。讀完了，拿到證書了嗎？還沒拿到。有收據嗎？有。證書呢？有。辦妥了才申請。好，申請了才辦妥。好，申請了，我可不可以怎樣辦妥，即是發放？因為申請了嘛，如果不申請的話，會怎樣？就錯過了。因為它下個年度會再計算，即是下年度2025年一月一日開始，因為你的收據是當年之前的，那就申請不到了。是啊，你現在申請是最後的了。因為我下個月就是截止日期了。哦，是嗎？但是你的收據已經是讀書時候的了。是啊。所以呢，就申請這個日子，它計算就計算這個日子。\n\n二十八元多。這麼便宜嗎？是啊。因為我們正在做特價！啊！是啊！我買了幾次。是啊，他們本身就不是很好的筆記，因為與質量有關的，但是我們說錯了，幾十元的那些東西，不是幾十元，是那個牌子的那些。幾百元的那些？你好像很厲害啊。好啊。除非你做校長，就可以控制到。啊！你不就是認識那些人嗎！是啊。其他老師都不是那麼容易堅持的嗎？是啊，為什麼？因為你小朋友像老師一樣，我年年都寫啦。是不是做完又做，不過後來老師說不要再做了。做回這些自己的，因為成本這麼貴，是啊！都是這樣說的！最主要是事實！這些老師不是很肯指正學生，真的很难教的嘛。當有什麼情況下，你控制不了學生，真的很難，煩心都煩不到我們！是啊，以前舊式的那些是，現在這個年代啊，所以呢我們教小朋友，那麼你就真的，不是開玩笑，是要逼。\n\n之前的那些不是這個價錢買的。\n其實標價是這個價錢。\n是啊，他們加了價錢。\n那時候我們入貨不是這個價錢。\n加了什麼時候？\n他們這個星期才加。\n所以囉，有些東西加幾十元。\n也有，也有的加幾元而已。\n最重要的是這個款式現在沒有了，就因為現在流行。\n我們這邊也沒有了，是吗？\n你們那邊應該差不多全部都沒有了現在。\n沒有了。\n這幾天呢，我們這邊啊，瘋狂地賣。\n啊，真的嗎？\n真的。\n沒關係，我們算是朋友價。\n\n那麼我…\n快點，令到自己全家好，那麼每個人都能够…\n삐——

## Refine Written Chinese (smoother sentence)
Using Gemini 1.5 pro

In [ ]:
def refine_writte_text(message, api_key):
    prompt = """
            Please refine the following Chinese text by making the sentence structure smoother and more coherent. 
            Reduce excessive punctuation, clarify ambiguous parts, and ensure natural readability while preserving the original meaning.
            Output in traditional Chinese characters (zh-Hant).
            """
    return gemini_prompt_call(message, api_key, prompt, temperature=1)

In [41]:
refined_written = refine_writte_text(raw_written, GEMINI_API_KEY)

In [42]:
refined_written

'根据去年的教师款项发放表格，流程就像这样，一条龙申请。现在的截止日期是今年的1月31日。我差不多忘了，大概今年的1月27号左右我收到了这些文件。他们每年1月1日都有假期。我再强调一次，每年都有这个申请。你需要在去年的1月1日至今年的1月31日之间完成整个课程，并持有收据和证书，才能回来领取款项。例如，（对某人说）“小姐，你读完了吗？” “读完了。” “拿到证书了吗？” “还没。” “有收据吗？” “有。” “证书呢？” “有。”  要办妥了才能申请。好，申请了才算办妥。好，申请了之后，我该如何办理发放款项呢？因为已经申请了，如果不申请会怎么样？就会错过。因为它下个年度会重新计算，也就是从2025年1月1日开始，因为你的收据是之前的，那就申请不到了。是的，你现在申请是最后的机会了。因为我下个月就是截止日期了。哦，是吗？但是你的收据已经是读书时候的了。是的。所以，申请的日期就是计算的日期。\n\n\n二十八元多。这么便宜吗？是的。因为我们正在做特价！啊！是的！我买了好几次了。是的，它们本身质量不是很好，因为跟质量有关，但是我们说错了，几十元的那些东西，不是指几十元的，是指那个牌子的。几百元的那些？你好像很懂行啊。还好。除非你做校长，才可以控制。啊！你不就是认识那些人吗！是的。其他老师都不是那么容易坚持的吗？是的，为什么？因为你像老师一样对待小朋友，我每年都写。是不是做完又做，不过后来老师说不要再做了。做回自己的，因为成本太高，是的！都是这样说的！最主要是事实！这些老师不是很肯指正学生，真的很难教。当有些情况下，你控制不了学生，真的很棘手，烦都烦不到我们！是的，以前旧式的那些是，现在这个年代啊，所以我们教小朋友，真的不是开玩笑，是要逼的。\n\n\n之前的那些不是这个价钱买的。\n其实标价就是这个价钱。\n是的，他们涨价了。\n我们那时候进货不是这个价钱。\n什么时候涨的？\n他们这个星期才涨。\n所以，有些东西涨了几十元。\n也有的只涨了几元。\n最重要的是这个款式现在没有了，就因为现在流行。\n我们这边也没有了，是吗？\n你们那边现在应该差不多全部都没有了。\n没有了。\n这几天，我们这边卖疯了。\n啊，真的吗？\n真的。\n没关系，我们算你朋友价。\n\n\n那么我…\n快点，让全家都好，那么每个人都能…\n삐——\n他帮了小朋友这么多，真的学到东西了。是的，所以

## Summarize using Gemini 1.5 Pro

In [43]:
def summarize_refined_written(message, api_key, sum_lvl):
    prompt = f"""Summarize the given Chinese text in Traditional Chinese (zh) at summarization level {sum_lvl}, where 1 is the most concise and 5 is the most detailed, and 0 is in bullet points with sectioned format. Retain key information and context while ensuring clarity, coherence, and fluency."""
    return gemini_prompt_call(message, api_key, prompt)

In [44]:
def summarize_refined_written(message, api_key, sum_lvl, bullet_points=False):
    if bullet_points:
        prompt = f"""Summarize the given Chinese text and output in Traditional Chinese (zh) markdown format with bullet points and sectioned format. Use detail level {sum_lvl}, where 1 is the most concise and 5 is the most detailed. Retain key information and context while ensuring clarity, coherence, and fluency."""
    else:
        prompt = f"""Summarize the given Chinese text and output in Traditional Chinese (zh) markdown format at summarization level {sum_lvl}, where 1 is the most concise and 5 is the most detailed. Retain key information and context while ensuring clarity, coherence, and fluency."""
    
    return gemini_prompt_call(message, api_key, prompt)

In [45]:
# Test
summarized_text = summarize_refined_written(refined_written, GEMINI_API_KEY, sum_lvl=2, bullet_points=True)

In [46]:
print(summarized_text)

## 教師款項申請與發放

* **申請流程與截止日期:**  款項申請流程簡便，截止日期為今年1月31日。文件約在1月27日左右收到。
* **申請資格:** 需在去年1月1日至今年1月31日期間完成指定課程，並持有收據和證書。
* **逾期後果:**  若逾期未申請，款項將作廢，因下個年度(2025年1月1日開始)會重新計算。目前是最後申請機會。
* **申請與發放:**  申請後即可辦理發放款項。


## 教學挑戰與特價商品

* **教學困境:**  現今學生難以管教，老師難以堅持像以往一樣的教學方式，成本也高。學生指正不易，控制困難。
* **特價商品:**  某商品正在特價，雖然品質一般，但價格便宜。進貨價與標價不同，最近才漲價，部分款式已售罄。將以朋友價出售。


## 學校活動與經費

* **招生不足:**  學校課程招生不足，目前只有八人，需想辦法增加人數。
* **經費運用:**  討論如何運用經費，例如英文教學法培訓，每人分攤費用約一千元。建議多討論與學校教法和教師法相關的運用方式，避免浪費。
* **資料提交:**  截止日期已到，收到的真實資料很少。已申請的費用、已完成的課程，以及相關單據和證書，請盡快提交申請。


## 網上分享與保護、聯校活動

* **網上分享:**  第一個報告關於保護和網上分享，不算成功。軟體會將老師的資料返還給老師和學生。將證書交給老師記錄，並自行整理數目。
* **聯校活動:**  聯校同行世界活動，需要正式的出入記錄翻譯版本或PPGT。四月將舉辦全港性的实习比赛，即使只有一人參與也會頒發證書。暫定4月12日在油尖旺舉行。


## 幼兒比賽與教學準備

* **幼兒比賽:**  K1、K2、K3都參與國際比賽，比賽日期為6月7日，在聯校活動之後。KG四已搞定，之前的比賽已按正常題目進行。
* **教學準備:**  影片可供參考，例如兩歲幼兒跑步的影片。準備教材，例如幾百字的文章，可在開學後撰寫。


## 教學平台與家長溝通

* **教學平台:**  討論iOS平台的使用，以及電子卡的使用情況。已使用三年，今年是第四年。
* **學生證管理:**  提醒學生不要將學生證掛在書包上，容易遺失。K2學生常將卡片插在口袋，建議使用布袋裝卡片。
* **家長溝通:**  處理眉機問題，以及良好的教育理念。



In [47]:
# Save as .md file
with open(f"text_output/{file_name}_summary.md", "w") as file:
    file.write(summarized_text)

print("Markdown file saved successfully.")

Markdown file saved successfully.


In [49]:
from IPython.display import display, Markdown

with open(f"text_output/{file_name}_summary.md", "r") as file:
    content = file.read()

display(Markdown(content))

## 教師款項申請與發放

* **申請流程與截止日期:**  款項申請流程簡便，截止日期為今年1月31日。文件約在1月27日左右收到。
* **申請資格:** 需在去年1月1日至今年1月31日期間完成指定課程，並持有收據和證書。
* **逾期後果:**  若逾期未申請，款項將作廢，因下個年度(2025年1月1日開始)會重新計算。目前是最後申請機會。
* **申請與發放:**  申請後即可辦理發放款項。


## 教學挑戰與特價商品

* **教學困境:**  現今學生難以管教，老師難以堅持像以往一樣的教學方式，成本也高。學生指正不易，控制困難。
* **特價商品:**  某商品正在特價，雖然品質一般，但價格便宜。進貨價與標價不同，最近才漲價，部分款式已售罄。將以朋友價出售。


## 學校活動與經費

* **招生不足:**  學校課程招生不足，目前只有八人，需想辦法增加人數。
* **經費運用:**  討論如何運用經費，例如英文教學法培訓，每人分攤費用約一千元。建議多討論與學校教法和教師法相關的運用方式，避免浪費。
* **資料提交:**  截止日期已到，收到的真實資料很少。已申請的費用、已完成的課程，以及相關單據和證書，請盡快提交申請。


## 網上分享與保護、聯校活動

* **網上分享:**  第一個報告關於保護和網上分享，不算成功。軟體會將老師的資料返還給老師和學生。將證書交給老師記錄，並自行整理數目。
* **聯校活動:**  聯校同行世界活動，需要正式的出入記錄翻譯版本或PPGT。四月將舉辦全港性的实习比赛，即使只有一人參與也會頒發證書。暫定4月12日在油尖旺舉行。


## 幼兒比賽與教學準備

* **幼兒比賽:**  K1、K2、K3都參與國際比賽，比賽日期為6月7日，在聯校活動之後。KG四已搞定，之前的比賽已按正常題目進行。
* **教學準備:**  影片可供參考，例如兩歲幼兒跑步的影片。準備教材，例如幾百字的文章，可在開學後撰寫。


## 教學平台與家長溝通

* **教學平台:**  討論iOS平台的使用，以及電子卡的使用情況。已使用三年，今年是第四年。
* **學生證管理:**  提醒學生不要將學生證掛在書包上，容易遺失。K2學生常將卡片插在口袋，建議使用布袋裝卡片。
* **家長溝通:**  處理眉機問題，以及良好的教育理念。


## 課程規劃與教案撰寫

* **課程規劃:**  K1、K2、K3課程規劃，K2是持續的每周主題，其他年級則是同一時間開始和結束同一主題。好處是方便處理指定活動和新主題，家長也更容易理解。
* **教案撰寫:**  教案撰寫需包含真實的教學活動，例如高班主任的練習。建議將其他活動也加入教案，使教案更貼近實際教學情況。


## 學校活動與行政安排

* **學校活動:**  新年假期活動、新春填色、生日會、親子活動等，需發出通知給家長。每月第一個星期五舉辦生日會。全校性活動例如水果日，需參考校本規劃。
* **行政安排:**  主任安排普通話比賽、布置活動、拍戲、訪問講座等，需自行報名。新家庭的參與安排也需考慮。


## 其他

*  討論寫意和寫生的區別，以及Grade示範功課。
*  會議已安排好，假期需準備功課和教材。
*  討論教師評分和出卷事宜。
*  家長活動的安排和規劃。
*  討論活動標籤和Facebook的關係。
*  K1和K3的計算數不平均問題。
*  活動數量和特色的要求。
*  理念分享和下一年工作的規劃。


This summary aims for detail level 2, balancing conciseness with necessary context.  It uses bullet points and sections for clarity and easy navigation.  It retains key information and presents it in a coherent and fluent manner in Traditional Chinese.
